# 머신러닝 기반 영화 리뷰 감성 분석
1. 데이터 준비 (전처리된 데이터)
2. 특징 벡터 추출 (TF-IDF)
3. 머신러닝 모델별 학습 및 평가
4. 영화 리뷰 긍부정 판단

## 1.데이터 준비 (전처리된 데이터)

In [1]:
data_filename = 'Korean_movie_reviews_2016.csv'

# 데이터 로딩
import pandas as pd
review_df=pd.read_csv('./data/'+data_filename)
review_df.head()

,review,label
0,부산 행 때문 너무 기대하고 봤,0
1,한국 좀비 영화 어색하지 않게 만들어졌 놀랍,1
2,조금 전 보고 왔 지루하다 언제 끝나 이 생각 드,0
3,평 밥 끼 먹자 돈 니 내고 미친 놈 정신사 좀 알 싶어 그래 밥 먹다 먹던 숟가락...,1
4,점수 대가 과 이 엑소 팬 어중간 점수 줄리 없겠 클레멘타인 이후 최고 평점 조작 ...,0


In [2]:
# 입력 데이터와 정답 데이터 추출
review_list = list(review_df.review)
label_list=list(review_df.label)
len(review_list), len(label_list)

(165384, 165384)

In [3]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
review_train, review_test, label_train, label_test = train_test_split(review_list, label_list, test_size=0.1)
len(review_train), len(review_test), len(label_train), len(label_test)

(148845, 16539, 148845, 16539)

## 2. 특징 벡터 추출 : TF-IDF

In [4]:
# 한국어 감성 분석용 tokenizer 정의
from konlpy.tag import Okt
def my_tokenizer(text):
    t = Okt()
    my_tags = ['Noun', 'Verb', 'Adjective']
    return [word for word, tag in t.pos(text) if tag in my_tags]

In [5]:
# 최대 단어 수 1000개
# 학습 데이터로 Vectorizer 생성 및 학습 데이터 특징 벡터 추출
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(tokenizer=my_tokenizer, max_features=1000)
X_train = tv.fit_transform(review_train)



C:\Users\User\anaconda3\envs\deepnlpenv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
# 테스트 데이터 특징 벡터 추출
X_test = tv.transform(review_test)
X_test.shape

(16539, 1000)

In [7]:
# 정답 데이터 변환 (np.array)
import numpy as np
Y_train = np.array(label_train)
Y_test = np.array(label_test)
Y_train.shape, Y_test.shape

((148845,), (16539,))

In [8]:
# 데이터 일부 확인
Y_train[:10]

array([1, 0, 1, 1, 0, 1, 1, 0, 0, 0])

## 3. 머신러닝 모델별 학습 및 평가
* 의사결정 트리
* 랜덤포레스트
* 나이브 베이즈
* 로지스틱 회귀 분석
* SVM
* Perceptron

In [9]:
# 머신러닝 모델별 학습 성능 평가 결과 저장 준비
import pandas as pd
score_df = pd.DataFrame(columns=['train', 'test'])

## 3.1 의사결정 트리 (Decision Tree)

In [10]:
# 학습
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=100)
dtc.fit(X_train, Y_train)

DecisionTreeClassifier(random_state=100)

In [11]:
# 성능 평가
train_score = dtc.score(X_train, Y_train)
test_score = dtc.score(X_test, Y_test)
print(train_score, test_score)

0.9811213006819174 0.8018018018018018


In [12]:
# 평가 결과 score_df에 추가
score_df.loc['DecisionTree'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802


## 3.2 랜덤 포레스트 (Random Forrest)

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X_train, Y_train)

RandomForestClassifier(n_jobs=-1)

In [14]:
# 성능 평가
train_score = rf.score(X_train, Y_train)
test_score = rf.score(X_test, Y_test)
print(train_score, test_score)

0.9811078638852497 0.8530140879134168


In [15]:
# 평가 결과 score_df에 추가
score_df.loc['RandomForest'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802
RandomForest,0.981108,0.853014


## 3.3 나이브 베이즈 (Naive Baysian)

In [16]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, Y_train)

MultinomialNB()

In [17]:
# 성능 평가
train_score = mnb.score(X_train, Y_train)
test_score = mnb.score(X_test, Y_test)
print(train_score, test_score)

0.853364237965669 0.8534373299473971


In [18]:
# 평가 결과 score_df에 추가
score_df.loc['NaiveBayes'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802
RandomForest,0.981108,0.853014
NaiveBayes,0.853364,0.853437


## 3.4 로지스틱 회귀 분석 (Logistic Regression)

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train, Y_train)

LogisticRegression(solver='liblinear')

In [20]:
# 성능 평가
train_score = lr.score(X_train, Y_train)
test_score = lr.score(X_test, Y_test)
print(train_score, test_score)

0.8622661157580033 0.8602092024910817


In [21]:
# 평가 결과 score_df에 추가
score_df.loc['LogisticRegression'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802
RandomForest,0.981108,0.853014
NaiveBayes,0.853364,0.853437
LogisticRegression,0.862266,0.860209


## 3.5 SVM (Support Vector Machine)

In [22]:
from sklearn.svm import LinearSVC
svc = LinearSVC(verbose=True)
svc.fit(X_train, Y_train)

[LibLinear]

LinearSVC(verbose=True)

In [23]:
# 성능 평가
train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)
print(train_score, test_score)

0.86182270146797 0.859604571013967


In [24]:
# 평가 결과 score_df에 추가
score_df.loc['SVM'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802
RandomForest,0.981108,0.853014
NaiveBayes,0.853364,0.853437
LogisticRegression,0.862266,0.860209
SVM,0.861823,0.859605


## 3.6 Perceptron

In [25]:
from sklearn.linear_model import Perceptron

perc = Perceptron()
perc.fit(X_train, Y_train)

Perceptron()

In [26]:
# 성능 평가
train_score = perc.score(X_train, Y_train)
test_score = perc.score(X_test, Y_test)
print(train_score, test_score)

0.8046491316470153 0.8013785597678215


In [27]:
# 평가 결과 score_df에 추가
score_df.loc['Perceptron'] = [train_score, test_score]
score_df

,train,test
DecisionTree,0.981121,0.801802
RandomForest,0.981108,0.853014
NaiveBayes,0.853364,0.853437
LogisticRegression,0.862266,0.860209
SVM,0.861823,0.859605
Perceptron,0.804649,0.801379


## 3.7 성능 비교

In [28]:
# 평가 결과 저장 데이터 프레임 확인
score_df.sort_values(by='test', ascending=False)

,train,test
LogisticRegression,0.862266,0.860209
SVM,0.861823,0.859605
NaiveBayes,0.853364,0.853437
RandomForest,0.981108,0.853014
DecisionTree,0.981121,0.801802
Perceptron,0.804649,0.801379


## 4. 영화 리뷰 긍부정 판단
* 학습된 모델 중 선택하여 활용

In [ ]:
# 영화 리뷰감성 분석용 tokenizer 정의


In [30]:
review = '영화가 재미있다'

# 텍스트 전처리
tokens=my_tokenizer(review)
tokens=" ".join(tokens)
print(tokens)
# 특징 벡터 추출
X = tv.transform([tokens])
# 예측
pred = svc.predict(X)
# 예측 결과 출력
result = '긍정' if pred >= 0.5 else '부정'
print(result)

영화 재미있다
긍정


In [46]:
# 함수로 만들기
def semtiment_analysis(review):
    # 텍스트 전처리
    tokens=my_tokenizer(review)
    tokens=" ".join(tokens)
    print(tokens)
    # 특징 벡터 추출
    X = tv.transform([tokens])
    # 예측
    pred = svc.predict(X)
    # 예측 결과 출력
    result = '긍정' if pred >= 0.5 else '부정'
    return result

In [47]:
# 함수 테스트
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ',
    '하품만 나온다',
    '이 영화 핵노잼 ㅠㅠ',
    '이딴게 영화냐 ㅉㅉ',
    '와 개쩐다',
    '감독 뭐하는 놈이냐',
    '정말 세계관 최강자들의 영화다'
]

for review in reviews:
    result = semtiment_analysis(review)
    print(f'{review} -> {result}')

이 영화 개꿀잼
이 영화 개꿀잼 ㅋㅋㅋ -> 부정
하품 나온다
하품만 나온다 -> 긍정
이 영화 핵 노잼
이 영화 핵노잼 ㅠㅠ -> 부정
게 영화
이딴게 영화냐 ㅉㅉ -> 부정
와 개 쩐다
와 개쩐다 -> 긍정
감독 뭐 하는 놈
감독 뭐하는 놈이냐 -> 부정
정말 세계관 최강 자 영화
정말 세계관 최강자들의 영화다 -> 부정


In [ ]:
# 문장을 입력 받아서 긍부정 판단 
